In [1]:
from pyspark.sql import functions as F

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1592075371796_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
input_s3 = 's3://udacity-dend/'
output_s3 = 's3://angelodatalake/sparkify/'

song_data = 'song_data/*/*/*/*.json'
log_data = 'log_data/*/*/*.json'

ex_song = 'song_data/A/B/C/TRABCEI128F424C983.json'
ex_log = 'log_data/2018/11/2018-11-13-events.json'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Testing with a sample

In [3]:
df_song_sample = spark.read.json(input_s3+ex_song)
df_song_sample.printSchema()

df_log_sample = spark.read.json(input_s3+ex_log)
df_log_sample.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: string (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: string (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = 

## Wrinting to my S3

In [4]:
df_song_sample.write.parquet(output_s3 + ex_song.replace('.json', '.parquet'),mode="overwrite")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Read from my S3

In [5]:
df_song_sample_my = spark.read.parquet(output_s3 + ex_song.replace('.json', '.parquet'))
df_song_sample_my.printSchema()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: string (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: string (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)

3.97s to 2.64s to read the sample <br>
Time dropped a lot when reading from my S3 as the cluster and S3 are in the same location

## Writing with partitions to S3

In [6]:
df_song_sample.write.partitionBy("artist_id","song_id").mode("append").parquet(output_s3 + ex_song.replace('.json', '.parquet'),mode="overwrite")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Clone S3

In [7]:
df_song = spark.read.json(input_s3+song_data)
df_song.write.partitionBy("artist_id").mode("append").parquet(output_s3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Reading all the data from my S3

Creating a dimension using Spark SQL vs processing as dataframe

In [27]:
df = spark.read.parquet(output_s3) # if you include an * at end of your path name, the partitioning variables will be dropped.
df.persist()
df.show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+--------------------+----------------+--------------------+---------+---------+------------------+--------------------+----+------------------+
|artist_latitude|     artist_location|artist_longitude|         artist_name| duration|num_songs|           song_id|               title|year|         artist_id|
+---------------+--------------------+----------------+--------------------+---------+---------+------------------+--------------------+----+------------------+
|       38.89086|Delaney: Ponotoc ...|       -90.18383|Delaney & Bonnie ...|208.61342|        1|SONPQTJ12A58A77889|Only You Know And...|1970|AR9SIJX1187FB4771F|
|        43.8586|Dubioza kolektiv ...|         18.4295|    Dubioza kolektiv|318.85016|        1|SOLPTHV12AC9073057|           Marijuana|2006|ARTHJGQ1187FB42F1F|
+---------------+--------------------+----------------+--------------------+---------+---------+------------------+--------------------+----+------------------+
only showing top 2 rows

All the process ran in less than 30s

In [24]:
df.createOrReplaceTempView('ext_song')
dim_song = spark.sql("""
        SELECT DISTINCT song_id, title, artist_id, year, duration
        FROM ext_song
""")
dim_song.show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------------+------------------+----+---------+
|           song_id|               title|         artist_id|year| duration|
+------------------+--------------------+------------------+----+---------+
|SONXXYP12A81C238EE|Don't Waste My Ti...|ARZNYC61187B9958CF|1985|324.85832|
|SORAUDC12A6D4F7273|The Debt Is Settl...|ARJF4E41187FB499FF|1978|233.11628|
+------------------+--------------------+------------------+----+---------+
only showing top 2 rows

# DataFrames

In [25]:
dim_song2= df.select(['song_id', 'title', 'artist_id', 'year', 'duration']).dropDuplicates()
dim_song2.show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------------+------------------+----+---------+
|           song_id|               title|         artist_id|year| duration|
+------------------+--------------------+------------------+----+---------+
|SOOORVX12A6D4FA612|Gimme A Chance  (...|ARKDZJP1187B98DD0C|2002|230.89587|
|SOVMCQA12AF72A3411|Chillin With My B...|ARBS1OC1187FB44CAF|2004|236.48608|
+------------------+--------------------+------------------+----+---------+
only showing top 2 rows

In [26]:
print('N° Rows: ', df.count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

N° Rows:  14896